# PROYECTO PYTHON CORTE 2 - 2019-2
## Integrantes
### - Cristian Duran Monsalve
### - Luis Carlos Jordan Hurtado
### - Kelly Fernanda Vásquez Zapata

## DECLARACIÓN DE VARIABLES Y DATOS PÚBLICOS

In [1]:
# Librerias que utiliza este aplicativo FLASK.
from flask import Flask, render_template, request, make_response, escape, url_for, redirect, flash, Response
from flask_debug import Debug
# Librerias que utiliza el Web scraper
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import csv
import mysql.connector as mariadb
import hashlib
# Libreria para convertir a imagen
import io
app = Flask(__name__)
app.secret_key = '123456'

csv_path = './static/futbol.csv'
csv_obj = csv.DictReader(open(csv_path, 'r'))
csv_list = list(csv_obj)

url = 'https://resultados.as.com/resultados/futbol/primera_femenina/clasificacion/'

def Conexion():
    return mariadb.connect(host='localhost', port='3306', user='root', password='123456', database='proyecto_python2')

def create_graph(path, function):
    figure = plt.figure(figsize=(10,8))
    function()
    plt.title('Lista de Equipos de Futbol Liga Femenina Española')
    plt.xlabel('Equipos')
    plt.ylabel('Puntajes')
    plt.axis('tight')
    plt.xticks(rotation='vertical') 
    figure.savefig('./static/images/graficas/' + path)
    # fig.savefig('./static/images/graficas/lineal.png', bbox_inches='tight', pad_inches=0)

## DECLARACIÓN DE RUTAS

### INDEX - PÁGINA PRINCIPAL

In [2]:
@app.route("/")
def index():
    return render_template("index.php")

### FUNCIONES WEBSCRAPING

In [3]:

@app.route("/menu_scraper")
def menu_scraper():
    return  render_template('menu_scraper.php')

@app.route("/webscraper_datos")
def webscraper_datos():
    equipos = []
    nequipos = WebScraper()
    partidos = []
    npartidos = WebScraper()
    lista = []
    eq = nequipos[0].values.tolist()
    pa = nequipos[1].values.tolist()
    total=len(eq)
    for x in range(total):
        lista.append([eq[x], pa[x]])
    
    webscrapertocsv = pd.DataFrame([npartidos], columns= ['Nombre Equipo', 'Partidos'])

    #Exportamos a csv
    webscrapertocsv.to_csv('./static/posiciones_futbol_femenino.csv', index = False)
    
    return render_template('webscraper_datos.php', list_obj=lista )

def WebScraper():
    page =requests.get(url).text
    soup = (BeautifulSoup(page, 'lxml'))
    
    equipos = []
    
    equipos_futbol = soup.find_all('span', attrs={'class':'nombre-equipo'})
    count = 0
    for i in equipos_futbol:
        if count < 6:
            equipos.append(i.text)
        else:
            break
        count += 1
        
    nequipos = pd.DataFrame(equipos)
    nequipos.head(6)
    
    partidos = []
    
     #Puntos por partidos jugados
    partido = soup.find_all('td', class_='destacado') 

    count = 0
    for i in partido:
        if count < 6:
            partidos.append(i.text)
        else:
            break
        count += 1    
        
    npartidos = pd.DataFrame(partidos)
    
    return nequipos, npartidos 

### FUNCIONES CVS

In [ ]:
@app.route("/menu_csv")
def menu_csv():
    return render_template("menu_csv.php")

@app.route("/csv")
def csv():
    datos = pd.read_csv('./static/futbol.csv')
    total = len(datos)
    return  render_template('datos_csv.php', object_list=datos, total=total)   
    
@app.route("/graficas_csv")
def graficas_csv():
    datos = pd.read_csv('./static/futbol.csv')
    df = pd.DataFrame(datos)
    grouped = df.groupby('Nombre Equipo')['Puntos de Clasificacion'].sum()
    create_graph('lineas_csv.png', lambda: plt.plot(df['Puntos de Clasificacion']))
    create_graph('barras_csv.png', lambda: plt.bar(grouped.index.tolist(), grouped.values.tolist()))
    create_graph('columnas_csv.png', lambda: plt.barh(grouped.index.tolist(), grouped.values.tolist()))
    create_graph('pie_csv.png', lambda: plt.pie(df['Puntos de Clasificacion'], labels=df['Nombre Equipo'], autopct="%0.1f %%", shadow=True))
    return  render_template('graficas_csv.php')  


### FUNCIONES BASES DE DATOS

In [ ]:
@app.route("/menu_bd")
def menu_bd():
    return render_template("menu_bd.php")

@app.route("/basedatos")
def basedatos():
    datos_db = []
    data = pd.read_sql('SELECT nombreequipos, puntuacion  FROM proyecto2', Conexion())
    total = len(data['nombreequipos'])
    for x in range(total):
        datos_db.append([data['nombreequipos'][x], data['puntuacion'][x]])
    return render_template('datos_sql.php', db_datos=datos_db)


@app.route("/graficas_bd")
def graficas_bd():
    data = pd.read_sql('SELECT substr(nombreequipos, 1, 7) as nombreequipos, sum(puntuacion) as puntuacion FROM proyecto2 group by nombreequipos', Conexion())
    histo = pd.read_sql(' select puntuacion from proyecto2', Conexion())
    create_graph('lineal.png', lambda: plt.plot(data['nombreequipos'], data['puntuacion']))
    create_graph('barras.png', lambda: plt.bar(data['nombreequipos'], data['puntuacion']))
    create_graph('columnas.png', lambda: plt.barh(data['nombreequipos'], data['puntuacion']))
    create_graph('porcentaje.png', lambda: plt.pie(data['puntuacion'], labels=data['nombreequipos'], autopct="%0.1f %%"))
    create_graph('histo.png', lambda: plt.hist(histo['puntuacion'], bins=11))
    return  render_template('graficas_bd.php')


### FINAL

In [ ]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Oct/2019 23:19:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2019 23:19:17] "GET /menu_csv HTTP/1.1" 200 -
